In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
import folium

In [3]:
df=pd.read_csv('1_17_merged.csv')

In [4]:
pd.set_option('display.max_columns', None)


In [5]:
df=pd.DataFrame(df)
df.head()

,Unnamed: 0,inspection_date,inspector,number_of_violations,violations,zip_code,house_numbers,categories,transactions,distance,review_count,longitude,latitude,name,sector,district,total_score,rating,price,establishment_x,establishment_y,address_x,address_y,trimmed_name_x,trimmed_name_y,clean_address_x,clean_address_y,address_distance,name_distance
0,0,2019-01-02,Cicily Martinez,3,"['22', '29', '45']",78212,2524,"['Italian', 'Desserts', 'Seafood']",['delivery'],4950.353662,100.0,-98.493780,29.453730,Capparelli's On Main,35,D,97,3.5,$$,capparellis on main,Capparelli's On Main,2524 main av n,2524 n main ave,capparellis main,capparellis main,main,n maine,3,0
1,1,2019-02-18 00:00:00,RYAN DAVIS,2,"['40', '45']",78231,14510,['Pizza'],['delivery'],9486.595668,36.0,-98.544347,29.575679,Marco's Pizza,9,A,98,3.0,$,marcos pizza,Marco's Pizza,14510 military hy nw,14510 nw military hwy,marcos pizza,marcos pizza,military hwy,nw military hwy,3,0
2,2,2019-06-11,Cicily Martinez,5,"['9', '35', '37', '37', '45']",78201,1535,"['Mexican', 'Breakfast & Brunch']",['delivery'],3142.464101,62.0,-98.520600,29.466670,El Rafa's Cafe,35,D,94,4.0,$,el rafas cafe,El Rafa's Cafe,1535 hildebrand w,1535 w hildebrand ave,rafas cafe,rafas cafe,hildebrand,w hildebrande,3,0
3,3,2019-09-25,Nathan Balfour,0,[],78256,15900,['Chocolatiers & Shops'],[],14640.405701,14.0,-98.615048,29.592770,Godiva Chocolatier,3,A,100,4.5,$$,godiva chocolatier #,Godiva Chocolatier,15900 la cantera parkway,15900 la cantera pkwy,godiv chocolatier #,godiv chocolatier,la cantera parkway,la cantera pkwy,3,2
4,4,2019-06-12,Sumner Bumgardner,5,"['21', '29', '29', '30', '39']",78215,200,"['Italian', 'Pizza']",['delivery'],7084.472909,401.0,-98.481876,29.437367,Tre Trattoria,19,C,98,4.0,$$,tre trattoria,Tre Trattoria,200 jones av w,200 w jones ave,tre trattoria,tre trattoria,jones,w jonese,3,0


In [6]:
df= df.drop(df[['transactions', 'price', 'establishment_x', 'establishment_y', 'address_x', 'address_y', 'trimmed_name_x', 'trimmed_name_y', 'clean_address_x', 'clean_address_y', 'address_distance', 'name_distance']], axis=1)

In [7]:
#Dropped all establishments with less than 22 reviews
df=  df[df['review_count'] >= 22] 
df.describe()

,Unnamed: 0,number_of_violations,zip_code,house_numbers,distance,review_count,longitude,latitude,sector,total_score,rating
count,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000
mean,1175.371941,5.103317,78220.850462,4857.146275,7838.718630,212.159326,-98.514251,29.489996,15.940729,93.977162,3.678358
std,710.891701,4.980941,15.134649,4769.731418,3066.936782,289.746588,0.058255,0.056054,9.163743,5.482043,0.669562
min,0.000000,0.000000,78201.000000,21.000000,811.199667,22.000000,-98.708442,29.354318,1.000000,56.000000,1.000000
25%,548.500000,2.000000,78209.000000,845.500000,5948.595993,52.000000,-98.564240,29.433553,8.000000,91.000000,3.500000
50%,1178.000000,4.000000,78216.000000,3011.000000,7986.163583,109.000000,-98.495517,29.492833,16.000000,95.000000,4.000000
75%,1792.500000,7.000000,78230.000000,8372.500000,9400.182938,255.000000,-98.480045,29.531368,22.000000,98.000000,4.000000
max,2416.000000,50.000000,78259.000000,22211.000000,20222.717722,2665.000000,-98.398119,29.661520,35.000000,100.000000,5.000000


In [8]:
#Dropped all establishments with less than 22 reviews
df=  df[df['review_count'] >= 22] 

In [9]:
#grouped health scores into 5 groups with a range of three
health_scores= {'health_scores' : []}
for x in df['total_score']:
#     print(x)
    if x > 95:
        health_scores['health_scores'].append('1')
    elif x> 90:
        health_scores['health_scores'].append('2')
    else:
        health_scores['health_scores'].append('3')       
#df
df['health_scores']= health_scores['health_scores']  


In [10]:
#df.isnull().sum()
#len(health_scores)
#len(df)
df.head()

,Unnamed: 0,inspection_date,inspector,number_of_violations,violations,zip_code,house_numbers,categories,distance,review_count,longitude,latitude,name,sector,district,total_score,rating,health_scores
0,0,2019-01-02,Cicily Martinez,3,"['22', '29', '45']",78212,2524,"['Italian', 'Desserts', 'Seafood']",4950.353662,100.0,-98.493780,29.453730,Capparelli's On Main,35,D,97,3.5,1
1,1,2019-02-18 00:00:00,RYAN DAVIS,2,"['40', '45']",78231,14510,['Pizza'],9486.595668,36.0,-98.544347,29.575679,Marco's Pizza,9,A,98,3.0,1
2,2,2019-06-11,Cicily Martinez,5,"['9', '35', '37', '37', '45']",78201,1535,"['Mexican', 'Breakfast & Brunch']",3142.464101,62.0,-98.520600,29.466670,El Rafa's Cafe,35,D,94,4.0,2
4,4,2019-06-12,Sumner Bumgardner,5,"['21', '29', '29', '30', '39']",78215,200,"['Italian', 'Pizza']",7084.472909,401.0,-98.481876,29.437367,Tre Trattoria,19,C,98,4.0,1
5,5,2019-02-18 00:00:00,RYAN DAVIS,0,[],78230,11003,"['Burgers', 'Fast Food']",7055.288004,42.0,-98.534699,29.555381,Whataburger,9,A,100,3.5,1


In [31]:
df.isnull().sum()

Unnamed: 0              0
inspection_date         0
inspector               0
number_of_violations    0
violations              0
zip_code                0
house_numbers           0
categories              0
distance                0
review_count            0
longitude               0
latitude                0
name                    0
sector                  0
district                0
total_score             0
rating                  0
health_scores           0
dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [12]:
#training / test split
X= df[['number_of_violations', 'review_count', 'rating']]
y= df['health_scores']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state = 123)

X_train.head()

,number_of_violations,review_count,rating
1541,14,44.0,2.0
1381,3,36.0,3.5
412,4,661.0,4.0
660,6,123.0,3.5
1165,0,75.0,4.5


In [13]:
# for classification you can change the algorithm to gini or entropy (information gain).  Default is gini.
clf = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=123)

In [14]:
#Fit the model to the training data

clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=123, splitter='best')

In [15]:
#Estimate health group

y_pred = clf.predict(X_train)
y_pred[1:5]

array(['1', '2', '2', '1'], dtype=object)

In [16]:
#Estimate the probability of a health group
y_pred_proba = clf.predict_proba(X_train)
y_pred_proba

array([[0.06451613, 0.32258065, 0.61290323],
       [0.6043956 , 0.37362637, 0.02197802],
       [0.37313433, 0.55970149, 0.06716418],
       ...,
       [0.91946309, 0.04697987, 0.03355705],
       [0.91946309, 0.04697987, 0.03355705],
       [0.10714286, 0.28571429, 0.60714286]])

In [17]:
confusion_matrix(y_train, y_pred)

array([[465, 109,  12],
       [ 71, 272,  63],
       [ 20,  61, 214]])

In [18]:
#Accuracy: number of correct predictions over the number of total instances that have been evaluated.
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.74


In [19]:
#Test Model

#Compute the accuracy of the model when run on the test data

print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

Accuracy of Decision Tree classifier on test set: 0.75


In [20]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           1       0.84      0.79      0.81       586
           2       0.62      0.67      0.64       406
           3       0.74      0.73      0.73       295

    accuracy                           0.74      1287
   macro avg       0.73      0.73      0.73      1287
weighted avg       0.74      0.74      0.74      1287



In [50]:
clf

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')